In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors

from numpy import array
from numpy import max
import numpy as np
import pandas as pd
import math
import folium

In [2]:
from download import download

# We choose daily data
url = "https://opendata.arcgis.com/datasets/e0f26f918c504608b10d128f65ce0efc_0.csv"
path_target = "datasets/Mesure_annuelle_Region_Occitanie_Polluants_Principaux.csv"
download(url, path_target, replace=False)

paris_df = pd.read_csv('PA13.csv', sep=';',
                          comment='#',
                          na_values="n/d",
                          converters={'heure': str})

Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.


## 1) Data treatment

In [3]:
occ_df = pd.read_csv(path_target)
occ_df = occ_df[occ_df['nom_poll'] == 'O3'] # only ozone

In [4]:
# On va traiter les données afin de rajouter une colonne de la date (jour)
occ_df['an'] = pd.to_datetime(occ_df['date_debut']).dt.to_period('Y')
# On rajoute une colonne pour avoir des valeurs standardisées
# Cela va servir à avoir une échelle des couleurs plus claire
occ_df['centrer_reduire'] = (occ_df[['valeur']] - np.mean(occ_df[['valeur']]))/ np.std(occ_df[['valeur']])

C:\Users\megan\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [5]:
occ_df.nom_station.unique()

array(['Toulouse-Jacquier Urbain', 'Peyrusse Vieille Rural',
       'Toulouse-Mazades Urbain', 'Albi Urbain',
       'Toulouse-Berthelot Urbain', 'Belesta en Lauragais Rural',
       'Montgiscard Urbain', 'Lourdes-Lapaca Urbain',
       'Tarbes-Dupuy Urbain', 'Castres-Travet Urbain', 'Millau Urbain',
       'Pamiers Urbain', 'Station Pedagogique',
       'Montpellier - Prés d Arènes Urbain',
       'Montpellier Nord - Périurbain', 'Montpellier Sud - Périurbain',
       'Biterrois-Narbonnais - Périurbain',
       'Agathois-Piscénois - Périurbain', 'Nîmes Sud - urbain',
       'La Calmette - Périurbain', 'Alès Cévennes - Pérurbain',
       'Saze - rural', 'Vallabrègues - rural', 'Perpignan Sud - urbain',
       'Saint Estève - périurbain', 'Rodez Urbain'], dtype=object)

In [6]:
occ_df.an.unique() # 362 jours, on fera varier notre map selon ces jours.

<PeriodArray>
['2018', '2017']
Length: 2, dtype: period[A-DEC]

In [7]:
import branca.colormap as cm

linear = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=-3, vmax=1
)
# couleurs

In [8]:
from ipywidgets import interact  # widget manipulation
from IPython.display import HTML

In [9]:
# rajoute Paris !!! 

def interactive_map(year = '2018'):
    
    occ_map = occ_df[occ_df['an'] == year]
    
    map_int = folium.Map(location = [43.9333, 2.15], 
                         zoom_start = 7.5, 
                         tiles = 'Stamen Terrain')
    
    for i in range(0,len(occ_map)):
        folium.Circle(
            location = [occ_map.iloc[i]['Y'], occ_map.iloc[i]['X']],
            popup = occ_map.iloc[i]['nom_station'],
            radius = occ_map.iloc[i]['valeur']*100,
            color = 'black',
            fill = True,
            fill_color = linear(occ_map.iloc[i]['centrer_reduire']),
            fill_opacity = 0.5,
            opacity = 0.4,
        ).add_to(map_int)
    
    return(map_int)

In [10]:
interact(interactive_map, month=occ_df.an.unique())

interactive(children=(Text(value='2018', description='year'), Output()), _dom_classes=('widget-interact',))

<function __main__.interactive_map(year='2018')>

## 4) Animation 


In [11]:
import matplotlib.animation as animation
# Comment obtient on une image de ce truc ?
# Animons tout ça...